# Clustering Images

Code from Chapter 6 *Clustering Images* of [**Programming Computer Vision**](http://programmingcomputervision.com/) by *Jan Erik Solem*.

Code has been modified to work with Python 3.x and also cleaned up use standarding aliases for numpy and matplotlib. A few other changes mostly in graphing code to more clearly show clusters.

In [ ]:
import os
import pickle

from PIL import Image, ImageDraw
from scipy.cluster.vq import kmeans, vq, whiten
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import imtools
import hcluster

## 6.1 *K*-Means Clustering

### SciPy Clustering example - 2D points

In [ ]:
class1 = 1.5 * np.random.randn(100,2)
class2 = np.random.randn(100, 2) + np.array([5, 5])
features = np.vstack((class1, class2))

In [ ]:
centroids, variance = kmeans(features, 2)

In [ ]:
code, distance = vq(features, centroids)

In [ ]:
plt.figure(figsize=(15,8))
ndx = np.where(code==0)[0]
plt.plot(features[ndx, 0], features[ndx, 1], '*')
ndx = np.where(code==1)[0]
plt.plot(features[ndx, 0], features[ndx, 1], 'r.')
plt.plot(centroids[:, 0], centroids[:, 1], 'go', markersize=15)
plt.axis('off')
plt.show()

In [ ]:
variance

In [ ]:
print(code)

In [ ]:
print(distance)

### Clustering images

In [ ]:
fonts_imlist = imtools.get_imlist('data/a_selected_thumbs/')
imnbr = len(fonts_imlist)
im = np.array(Image.open(fonts_imlist[0]))
m, n = im.shape[0:2]
print(f"{imnbr} images of size ({m}, {n}) to be loaded")

Load model file

In [ ]:
with open('font_pca_modes.pkl', 'rb') as f:
    immean = pickle.load(f)
    V = pickle.load(f)

Create matrix to store flattened images and loaded images into it

In [ ]:
immatrix = np.array([np.array(Image.open(im)).flatten() for im in fonts_imlist], 'f')

Let's see what they look like

In [ ]:
plt.figure(figsize=(15, 4))
plt.gray()
for i in range(imnbr):
    plt.subplot(4,19,i+1)
    plt.imshow(immatrix[i].reshape(m, n))
    plt.axis('off')

Project on the 40 first PCs

In [ ]:
immean = immean.flatten()
projected = np.array([np.dot(V[:40], immatrix[i]-immean) for i in range(imnbr)])

*K*-means

In [ ]:
k = 3

In [ ]:
projected = whiten(projected)
centroids, distorted = kmeans(projected, k)
code, distance = vq(projected, centroids)

Plot clusters

In [ ]:
grid_size_for_k = {2:(2,1), 3:(2,2), 4:(2,2), 5:(3,2), 6:(3,3), 7:(4,2), 8:(4,2)}
ncol, nrow = grid_size_for_k[k]

In [ ]:
fig = plt.figure(figsize=(15, 8))
outer = gridspec.GridSpec(ncol, nrow, wspace=0.2, hspace=0.2)
plt.gray()
for i in range(k):
    ind = np.where(code==i)[0]
    inner = gridspec.GridSpecFromSubplotSpec(4, 10,
                    subplot_spec=outer[i], wspace=0.1, hspace=0.1)
    for j in range(np.minimum(len(ind), 40)):
        ax = plt.Subplot(fig, inner[j])
        ax.axis('off')
        ax.imshow(immatrix[ind[j]].reshape(m, n))
        if j == 0:
            ax.set_title(f"Cluster {i}")
        fig.add_subplot(ax)
plt.show()

### Visualizing the Images on Principal Components

Pick two principal components for visualization

In [ ]:
component1, component2 = 0, 1

In [ ]:
projected_2pc = np.array([np.dot(V[[component1, component2]], immatrix[i] - immean) for i in range(imnbr)])

In [ ]:
projected_2pc.shape

Create a new image with a white background

In [ ]:
h, w = 1200, 1200
img = Image.new('RGB', (w, h), (255, 255, 255))
draw = ImageDraw.Draw(img)

Draw axis

In [ ]:
draw.line( (0, h/2, w, h/2), fill=(255,0,0) )
draw.line( (w/2, 0, w/2, h), fill=(255,0,0) )

Scale coordinates to fit

In [ ]:
scale = abs(projected_2pc).max(0)
scaled = np.floor(np.array([ (p/scale) * ( w/2 - 20, h/2 - 20 ) + (w/2, h/2) for p in projected_2pc])).astype(int)

Paste thumbnail of each image

In [ ]:
for i in range(imnbr):
    nodeim = Image.open(fonts_imlist[i])
    nodeim.thumbnail((25, 25))
    ns = nodeim.size
    a = scaled[i][0] - ns[0] // 2
    b = scaled[i][1] - ns[1] // 2
    c = scaled[i][0] + ns[0] // 2 + 1
    d = scaled[i][1] + ns[1] // 2 + 1
    img.paste(nodeim, box=(a, b, c, d))

In [ ]:
img.save('pca_font.jpg')

In [ ]:
plt.figure(figsize=(8,10))
plt.imshow(img)
plt.axis('off')
plt.show()

### Clustering Pixels

Load image

In [ ]:
im = np.array(Image.open('data/empire.jpg'))

In [ ]:
plt.figure(figsize=(4,5))
plt.imshow(im)
plt.axis('off')
plt.show()

Divide image into steps * steps regions

In [ ]:
steps = 50
height = im.shape[0]
width = im.shape[1]
dx =  height // steps
dy = width // steps
print(f"Image size: ({height},{width}), dx={dx}, dy={dy}")

Compute color features for each region

In [ ]:
features = []
for x in range(steps):
    for y in range(steps):
        R = np.mean(im[x*dx:(x+1)*dx, y*dy:(y+1)*dy, 0])
        G = np.mean(im[x*dx:(x+1)*dx, y*dy:(y+1)*dy, 1])
        B = np.mean(im[x*dx:(x+1)*dx, y*dy:(y+1)*dy, 2])
        features.append([R, G, B])
features = np.array(features, 'f')

Cluster

In [ ]:
centroids, variance = kmeans(features, 3)
code, distance = vq(features, centroids)

In [ ]:
code.shape

Create image with cluster labels

In [ ]:
codeim = code.reshape(steps, steps)
codeim = np.array(Image.fromarray(codeim).resize((width,height), Image.NEAREST)).astype(np.double)

In [ ]:
plt.figure()
plt.imshow(codeim)
plt.axis('off')
plt.show()

## 6.2 Hierarchical Clustering

Create some 2D points

In [ ]:
class1 = 1.5 * np.random.randn(100,2)
class2 = np.random.randn(100, 2) + np.array([5, 5])
features = np.vstack((class1, class2))

Cluster points and extracts clusters using threshold of 5

In [ ]:
tree = hcluster.hcluster(features)
clusters = tree.extract_clusters(5)

Print clusters to console

In [ ]:
print(f"Number of clusters: {len(clusters)}")
for c in clusters:
    print(c.get_cluster_elements())

Note: values in each cluster correspond to index of point in features vector. Expect to have 2 clusters one with values less than 100 (corresponding to class1) and one with values greater than 100 (corresponding to class 2) 

### Clustering Images

In [ ]:
sunset_imlist = imtools.get_imlist('data/flickr-sunsets-small/')

Extract feature vector (8 bins per color channel)

In [ ]:
features = np.zeros([len(sunset_imlist), 512])
for i, f in enumerate(sunset_imlist):
    im = np.array(Image.open(f))
    
    h, edges = np.histogramdd(im.reshape(-1, 3), 8, normed=True, range=[(0,255), (0,255), (0,255)])
    features[i] = h.flatten()
    
tree = hcluster.hcluster(features)

Draw a dendogram

In [ ]:
hcluster.draw_dendrogram(tree, sunset_imlist, filename='sunset.pdf')

Visualize clusters with some (arbitrary) threshold

In [ ]:
clusters = tree.extract_clusters(0.23 * tree.distance)

for c in clusters:
    elements = c.get_cluster_elements()
    nbr_elements = len(elements)
    if nbr_elements > 3:
        plt.figure()
        for p in range(np.minimum(nbr_elements, 20)):
            plt.subplot(4, 5, p+1)
            im = np.array(Image.open(sunset_imlist[elements[p]]))
            plt.imshow(im)
            plt.axis('off')
plt.show()

Let's also draw a dendogram for the font images

In [ ]:
tree = hcluster.hcluster(projected)
hcluster.draw_dendrogram(tree, fonts_imlist, filename='fonts.jpg')

## 6.3 Spectral Clustering

Reusing projected font vectors again

In [ ]:
n = len(projected)

Compute distance matrix

In [ ]:
S = np.array([[np.sqrt(np.sum((projected[i] - projected[j])**2)) for i in range(n)] for j in range(n)], 'f')

Create Laplacian matrix

In [ ]:
rowsum = np.sum(S, axis=0)
D = np.diag(1 / np.sqrt(rowsum))
I = np.identity(n)
L = I - np.dot(D, np.dot(S, D))

Compute eigenvectors of L

In [ ]:
U, sigma, V = np.linalg.svd(L)

Create feature vector from k first eigenvectors by stacking eigenvectors as columns

In [ ]:
k = 5

In [ ]:
features = np.array(V[:k]).T

*K*-Means

In [ ]:
features = whiten(features)
centroids, distortion = kmeans(features, k)
code, distance = vq(features, centroids)

Plot clusters

In [ ]:
ncol, nrow = grid_size_for_k[k]
fig = plt.figure(figsize=(15, 8))
outer = gridspec.GridSpec(ncol, nrow, wspace=0.2, hspace=0.2)
plt.gray()
for i in range(k):
    ind = np.where(code==i)[0]
    inner = gridspec.GridSpecFromSubplotSpec(4, 10,
                    subplot_spec=outer[i], wspace=0.1, hspace=0.1)
    for j in range(np.minimum(len(ind), 39)):
        im = Image.open(fonts_imlist[ind[j]])
        ax = plt.Subplot(fig, inner[j])
        ax.axis('off')
        ax.imshow(np.array(im))
        if j == 0:
            ax.set_title(f"Cluster {i}")
        fig.add_subplot(ax)
plt.show()